In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import kagglehub

# Download data and create genre column for tracks using artists
def download_and_clean():

    path = kagglehub.dataset_download("yamaerenay/spotify-dataset-19212020-600k-tracks")
    artists_path = path + '/artists.csv'
    tracks_path = path + '/tracks.csv'

    artists_df = pd.read_csv(artists_path)
    tracks_df = pd.read_csv(tracks_path)

    artists_df = artists_df.dropna().drop_duplicates()
    tracks_df = tracks_df.dropna().drop_duplicates()

    artists_df = artists_df[artists_df['genres'].apply(lambda x: len(eval(x)) > 0 if isinstance(x, str) else False)]

    valid_artists = set(artists_df['name'])
    tracks_df = tracks_df[tracks_df['artists'].apply(lambda x: all(artist in valid_artists for artist in eval(x)) if isinstance(x, str) else False)]

    artist_to_genres = dict(zip(artists_df['name'], artists_df['genres']))

    def get_genres_for_track(artists_list):
        if isinstance(artists_list, str):
            try:
                artists_list = eval(artists_list)
                genres = [genre for artist in artists_list if artist in artist_to_genres for genre in eval(artist_to_genres[artist])]
                return list(set(genres))
            except:
                return []
        return []

    tracks_df['genres'] = tracks_df['artists'].apply(get_genres_for_track)

    artists_df.to_csv('/content/drive/MyDrive/nlp-proj/data/cleaned_data/cleaned_artists.csv', index=False)
    tracks_df.to_csv('/content/drive/MyDrive/nlp-proj/data/cleaned_data/cleaned_tracks.csv', index=False)

    return artists_df, tracks_df


In [ ]:
from data.cleaning import download_and_clean

artists, tracks = download_and_clean()

100%|██████████| 193M/193M [00:10<00:00, 18.4MB/s]

Extracting files...


In [ ]:
print(artists.head())

                         id  followers                      genres  \
45   0VLMVnVbJyJ4oyZs2L3Yl2       71.0          ['carnaval cadiz']   
46   0dt23bs4w8zx154C5xdVyl       63.0          ['carnaval cadiz']   
47   0pGhoB99qpEJEsBQxgaskQ       64.0          ['carnaval cadiz']   
48   3HDrX2OtSuXLW5dLR85uN3       53.0          ['carnaval cadiz']   
136  22mLrN5fkppmuUPsHx6i2G       59.0  ['classical harp', 'harp']   

                             name  popularity  
45   Las Viudas De Los Bisabuelos           6  
46              Los De Capuchinos           5  
47             Los “Pofesionales”           7  
48      Los Que No Paran De Rajar           6  
136                   Vera Dulova           3  


In [ ]:
print(tracks.head())

                       id                                name  popularity  \
2  07A5yehtSnoedViJAZkNnc  Vivo para Quererte - Remasterizado           0   
3  08FmqUhxtyLTn6pAh6bk45       El Prisionero - Remasterizado           0   
4  08y9GfoqCWfOGsKdwojr5e                 Lady of the Evening           0   
5  0BRXJHRNGQ3W4v9frnSfhu                           Ave Maria           0   
7  0IA0Hju8CAgYfV1hwhidBH                             La Java           0   

   duration_ms  explicit              artists                  id_artists  \
2       181640         0  ['Ignacio Corsini']  ['5LiOoJbxVSAMkBS2fUm3X2']   
3       176907         0  ['Ignacio Corsini']  ['5LiOoJbxVSAMkBS2fUm3X2']   
4       163080         0      ['Dick Haymes']  ['3BiJGZsyX9sJchTqcSA7Su']   
5       178933         0      ['Dick Haymes']  ['3BiJGZsyX9sJchTqcSA7Su']   
7       161427         0      ['Mistinguett']  ['4AxgXfD7ISvJSTObqm4aIE']   

  release_date  danceability  energy  ...  loudness  mode  speechiness  \


In [ ]:
# The next few cells create a multi-hot vector for storing the genre labels for modeling
from itertools import chain
all_genres = set(chain.from_iterable(tracks["genres"]))
len(all_genres)
# all_genres

4706

In [ ]:
genre_to_index = {genre: idx for idx, genre in enumerate(sorted(all_genres))}
# genre_to_index

In [ ]:
def multihot_vector(genres, index_dict):
    multihot = [0] * len(index_dict)
    for genre in genres:
        multihot[index_dict[genre]] = 1
    return multihot

In [ ]:
tracks["multi_hot_genres"] = tracks["genres"].apply(
    lambda genres: multihot_vector(genres, genre_to_index)
)
tracks["multi_hot_genres"]

,multi_hot_genres
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...
586667,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
586668,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
586669,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
586670,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
# Split the data into 80-10-10

from sklearn.model_selection import train_test_split

X = tracks[["name", "popularity", "duration_ms", "explicit", "release_date", "danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "time_signature"]]
y = tracks["multi_hot_genres"].tolist()

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=478)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=478)

In [ ]:
print(f"Train set size: {len(X_train)}")
print(f"Dev set size: {len(X_dev)}")
print(f"Test set size: {len(X_test)}")

Train set size: 399251
Dev set size: 49906
Test set size: 49907


In [ ]:
# Drop genres that have less than 100 occurences

import numpy as np

label_counts = np.sum(y_train, axis=0)
threshold =100
selected_labels = np.where(label_counts >= threshold)[0]

def filter_labels(y, selected_labels):
    return np.array([[y_sample[i] for i in selected_labels] for y_sample in y])

y_train = filter_labels(y_train, selected_labels)
y_dev = filter_labels(y_dev, selected_labels)
y_test = filter_labels(y_test, selected_labels)

filtered_genre_to_index = {genre: idx for idx, genre in enumerate(selected_labels)}
index_to_genre = {v: k for k, v in filtered_genre_to_index.items()}

print(f"Original number of labels: {len(label_counts)}")
print(f"Filtered number of labels: {len(selected_labels)}")


Original number of labels: 4706
Filtered number of labels: 1529


In [ ]:
# Drop rows with no positive labels

def remove_empty_labels(X, y):
    non_empty_indices = [i for i, labels in enumerate(y) if np.sum(labels) > 0]
    X_filtered = X.iloc[non_empty_indices].reset_index(drop=True)
    y_filtered = np.array([y[i] for i in non_empty_indices])
    return X_filtered, y_filtered

X_train, y_train = remove_empty_labels(X_train, y_train)
X_dev, y_dev = remove_empty_labels(X_dev, y_dev)
X_test, y_test = remove_empty_labels(X_test, y_test)

print(f"Train set size after filtering: {len(X_train)}")
print(f"Dev set size after filtering: {len(X_dev)}")
print(f"Test set size after filtering: {len(X_test)}")

Train set size after filtering: 386974
Dev set size after filtering: 48297
Test set size after filtering: 48376


In [ ]:
from transformers import RobertaModel, RobertaTokenizer
import torch
import torch.nn as nn
from transformers import TrainingArguments, Trainer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, jaccard_score, hamming_loss

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
# Tokenizing and preparing the feature for modeling
def tokenize_data(X):
    combined_text = X.apply(lambda row: " ".join(row.astype(str)), axis=1)

    return tokenizer(
        combined_text.tolist(),
        padding=True,
        truncation=True,
        max_length=64,
        return_tensors="pt"
    )

# Dataset preparation
class MultiLabelDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        tokenized = tokenize_data(X)

        self.input_ids = tokenized["input_ids"]
        self.attention_mask = tokenized["attention_mask"]
        self.labels = torch.tensor(y)

        token_lengths = [len(ids) for ids in self.input_ids]
        print(f"Token length range: {min(token_lengths)} to {max(token_lengths)}")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }

In [ ]:
train_dataset = MultiLabelDataset(X_train, y_train)
dev_dataset = MultiLabelDataset(X_dev, y_dev)
test_dataset = MultiLabelDataset(X_test, y_test)

Token length range: 64 to 64
Token length range: 64 to 64
Token length range: 64 to 64


In [ ]:
# RoBERTa, following similar to the model described in “Punk or Funk: Understanding the Performance of RoBERTa on Music Genre Classification.”
class MultiLabelRoBERTa(nn.Module):
    def __init__(self, num_labels):
        super(MultiLabelRoBERTa, self).__init__()
        self.roberta = RobertaModel.from_pretrained("roberta-base")
        self.classifier = nn.Sequential(
            nn.Linear(self.roberta.config.hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_labels)
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)

        logits = self.classifier(outputs.pooler_output)

        loss = None
        if labels is not None:
            criterion = nn.BCEWithLogitsLoss()
            loss = criterion(logits, labels.float())

        return {"loss": loss, "logits": logits}

# Metrics
def compute_metrics(pred, threshold_range=(0.05, 0.55, 0.05), prob=True):
    logits, labels = pred
    output = logits
    if prob:
        output = torch.sigmoid(torch.tensor(logits))

    # Find probability range
    prob_min = output.min().item()
    prob_max = output.max().item()
    prob_mean = output.mean().item()

    def prob_threshold_pred(output, threshold):
        if isinstance(output, np.ndarray):
            return (output > threshold).astype(int)
        return (output > threshold).int()

    def calc_metrics(labels, predictions):
        subset_accuracy = accuracy_score(labels, predictions)
        precision = precision_score(labels, predictions, average="micro")
        recall = recall_score(labels, predictions, average="micro")
        f1 = f1_score(labels, predictions, average="micro")
        jaccard = jaccard_score(labels, predictions, average='samples')
        hamming = hamming_loss(labels, predictions)

        return subset_accuracy, precision, recall, f1, jaccard, hamming

    threshold_results = {}
    for threshold in np.arange(*threshold_range):
        predictions = prob_threshold_pred(output, threshold)
        subset_accuracy, precision, recall, f1, jaccard, hamming = calc_metrics(labels, predictions)
        threshold_results[threshold] = {
            "accuracy": subset_accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "hamming": hamming,
            "jaccard": jaccard
        }

    # Find optimal probability threshold
    optimal_threshold = max(threshold_results, key=lambda t: threshold_results[t]["f1"])
    optimal_threshold_metrics = threshold_results[optimal_threshold]

    return {
        "prob_min": prob_min,
        "prob_max": prob_max,
        "prob_mean": prob_mean,
        "optimal_threshold": optimal_threshold,
        "optimal_threshold_accuracy": optimal_threshold_metrics["accuracy"],
        "optimal_threshold_precision": optimal_threshold_metrics["precision"],
        "optimal_threshold_recall": optimal_threshold_metrics["recall"],
        "optimal_threshold_f1": optimal_threshold_metrics["f1"],
        "optimal_threshold_hamming": optimal_threshold_metrics["hamming"],
        "optimal_threshold_jaccard": optimal_threshold_metrics["jaccard"],
    }


In [ ]:
# Initialize the model
def initialize_model():
    num_labels = len(y_train[0])
    model = MultiLabelRoBERTa(num_labels=num_labels)
    model.roberta.gradient_checkpointing_enable()
    return model
model = initialize_model()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.model_selection import ParameterSampler

# Hyperparameters; we are very limited in processing power, so only searching with low # of epochs
param_grid = {
    "learning_rate": [1e-5],
    "num_train_epochs": [5, 10],
    "per_device_train_batch_size": [32, 64, 128],
    "weight_decay": [0.01]
}

# Random Search, like in the Stanford Paper
n_trials = 5
param_samples = list(ParameterSampler(param_grid, n_iter=n_trials, random_state=478))

best_score = -np.inf
best_params = None
best_model = None

# Begin Search
for i, params in enumerate(param_samples):
    print(f"Trial {i+1}/{n_trials} with parameters: {params}")

    model = initialize_model()

    training_args = TrainingArguments(
        output_dir=f"./results/trial_{i}",
        eval_strategy="no",
        save_strategy="no",
        learning_rate=params["learning_rate"],
        per_device_train_batch_size=params["per_device_train_batch_size"],
        per_device_eval_batch_size=16,
        num_train_epochs=params["num_train_epochs"],
        weight_decay=params["weight_decay"],
        fp16=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        processing_class=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()

    # Evaluate
    metrics = trainer.evaluate(dev_dataset)
    f1 = metrics["eval_optimal_threshold_f1"]

    print(f"Trial {i+1} F1 Score: {f1}")
    print(metrics)

    # Track best model
    if f1 > best_score:
        best_score = f1
        best_params = params
        best_model = trainer
        trainer.save_model("/content/drive/MyDrive/nlp-proj/models")

# Output
print(f"Best F1 Score: {best_score}")
print(f"Best Parameters: {best_params}")

# Save the best model
best_model.save_model("/content/drive/MyDrive/nlp-proj/best-model")


Trial 1/5 with parameters: {'weight_decay': 0.01, 'per_device_train_batch_size': 64, 'num_train_epochs': 5, 'learning_rate': 1e-05}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.515600
1000,0.175000
1500,0.063200
2000,0.037600
2500,0.029800
3000,0.026100
3500,0.024000
4000,0.022100
4500,0.020900
5000,0.019700


Trial 1 F1 Score: 0.34491615640342227
{'eval_loss': 0.010151389054954052, 'eval_prob_min': 1.8954015246208655e-08, 'eval_prob_max': 0.9599877595901489, 'eval_prob_mean': 0.0018018056871369481, 'eval_optimal_threshold': 0.1, 'eval_optimal_threshold_accuracy': 0.043957181605482745, 'eval_optimal_threshold_precision': 0.3309308465252612, 'eval_optimal_threshold_recall': 0.36013567765087584, 'eval_optimal_threshold_f1': 0.34491615640342227, 'eval_optimal_threshold_hamming': 0.0033915258342710605, 'eval_optimal_threshold_jaccard': 0.21007722817861058, 'eval_runtime': 1036.3281, 'eval_samples_per_second': 46.604, 'eval_steps_per_second': 2.913, 'epoch': 5.0}
Trial 2/5 with parameters: {'weight_decay': 0.01, 'per_device_train_batch_size': 64, 'num_train_epochs': 10, 'learning_rate': 1e-05}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Step,Training Loss
500,0.513700
1000,0.175200
1500,0.064500
2000,0.038200
2500,0.030400
3000,0.026400
3500,0.024000
4000,0.022300
4500,0.021000
5000,0.020200


Trial 4 F1 Score: 0.4711043942467375
{'eval_loss': 0.007619387470185757, 'eval_prob_min': 1.8140579370693605e-14, 'eval_prob_max': 0.9986541271209717, 'eval_prob_mean': 0.0020383521914482117, 'eval_optimal_threshold': 0.2, 'eval_optimal_threshold_accuracy': 0.08549185249601425, 'eval_optimal_threshold_precision': 0.49697208498772466, 'eval_optimal_threshold_recall': 0.4477963305859155, 'eval_optimal_threshold_f1': 0.4711043942467375, 'eval_optimal_threshold_hamming': 0.0024927513788031066, 'eval_optimal_threshold_jaccard': 0.3168503805695805, 'eval_runtime': 1033.5406, 'eval_samples_per_second': 46.73, 'eval_steps_per_second': 2.921, 'epoch': 10.0}
Trial 5/5 with parameters: {'weight_decay': 0.01, 'per_device_train_batch_size': 32, 'num_train_epochs': 5, 'learning_rate': 1e-05}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.514000
1000,0.176100
1500,0.065100
2000,0.038500
2500,0.030500
3000,0.026600
3500,0.024400
4000,0.022800
4500,0.021500
5000,0.020600


Trial 5 F1 Score: 0.38740658637475145
{'eval_loss': 0.009128551930189133, 'eval_prob_min': 6.522847539880416e-11, 'eval_prob_max': 0.9844563603401184, 'eval_prob_mean': 0.0019633439369499683, 'eval_optimal_threshold': 0.15000000000000002, 'eval_optimal_threshold_accuracy': 0.05404062364121995, 'eval_optimal_threshold_precision': 0.40609073214553076, 'eval_optimal_threshold_recall': 0.37036612209896164, 'eval_optimal_threshold_f1': 0.38740658637475145, 'eval_optimal_threshold_hamming': 0.0029039036895550617, 'eval_optimal_threshold_jaccard': 0.24344145432089473, 'eval_runtime': 1041.5463, 'eval_samples_per_second': 46.37, 'eval_steps_per_second': 2.899, 'epoch': 5.0}
Best F1 Score: 0.4711043942467375
Best Parameters: {'weight_decay': 0.01, 'per_device_train_batch_size': 32, 'num_train_epochs': 10, 'learning_rate': 1e-05}


In [ ]:
#Evaluate on test
best_model.evaluate(test_dataset)